<a href="https://colab.research.google.com/github/mheloisagalvao/cnn-ia/blob/main/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import gdown
import zipfile
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Configurações
dataset_url = 'https://drive.google.com/uc?id=1y4pRBFouUBsA0RNLs7ZlR3CZF1OFvpYW'
dataset_zip = 'dataset.zip'
extracted_folder = 'dataset'
batch_size = 32
input_shape = (224, 224, 3)
classes = 4

# Função para baixar e extrair o dataset
def download_and_extract_dataset(url, zip_filename, extract_folder):
    gdown.download(url, zip_filename, quiet=False)
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall(extract_folder)
    os.remove(zip_filename)

# Baixa e extrai o dataset
download_and_extract_dataset(dataset_url, dataset_zip, extracted_folder)

Downloading...
From: https://drive.google.com/uc?id=1y4pRBFouUBsA0RNLs7ZlR3CZF1OFvpYW
To: /content/dataset.zip
100%|██████████| 278M/278M [00:05<00:00, 49.1MB/s]


In [4]:
# Configuração do gerador de imagens
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # divide automaticamente em treino e validação
)

# Carrega o conjunto de treinamento
train_generator = datagen.flow_from_directory(
    extracted_folder,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

# Carrega o conjunto de validação
validation_generator = datagen.flow_from_directory(
    extracted_folder,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 1600 images belonging to 4 classes.
Found 400 images belonging to 4 classes.


In [7]:
# Função para criar modelo CNN
def create_cnn_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=input_shape, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Função para pré-processar e carregar imagens, removendo as corrompidas
def load_images_with_error_handling(generator):
    while True:
        try:
            data, labels = next(generator)
            yield data, labels
        except (OSError, tf.errors.InvalidArgumentError):
            print("Error loading batch. Skipping...")
            pass

In [8]:
from tensorflow.keras.callbacks import EarlyStopping

# Criação do callback EarlyStopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

# Treina o primeiro modelo CNN
model1 = create_cnn_model()
history1 = model1.fit(load_images_with_error_handling(train_generator),
                      validation_data=load_images_with_error_handling(validation_generator),
                      epochs=10,
                      callbacks=[early_stopping])

Epoch 1/10
     16/Unknown - 40s 2s/step - loss: 32.3369 - accuracy: 0.3828Error loading batch. Skipping...
     40/Unknown - 98s 2s/step - loss: 14.7790 - accuracy: 0.4688Error loading batch. Skipping...
    106/Unknown - 248s 2s/step - loss: 5.8995 - accuracy: 0.7008Error loading batch. Skipping...
    126/Unknown - 306s 2s/step - loss: 5.0364 - accuracy: 0.7329Error loading batch. Skipping...
    181/Unknown - 432s 2s/step - loss: 3.6025 - accuracy: 0.7838Error loading batch. Skipping...
    252/Unknown - 596s 2s/step - loss: 2.6705 - accuracy: 0.8191Error loading batch. Skipping...
    267/Unknown - 631s 2s/step - loss: 2.5366 - accuracy: 0.8240Error loading batch. Skipping...
    334/Unknown - 785s 2s/step - loss: 2.0742 - accuracy: 0.8435Error loading batch. Skipping...
    387/Unknown - 918s 2s/step - loss: 1.8222 - accuracy: 0.8539Error loading batch. Skipping...
    456/Unknown - 1075s 2s/step - loss: 1.5772 - accuracy: 0.8660Error loading batch. Skipping...
    489/Unknown - 

KeyboardInterrupt: ignored

In [9]:
# Avaliação do modelo no conjunto de validação
evaluation_result = model1.evaluate(validation_generator)
print(f'Loss on validation set: {evaluation_result[0]}')
print(f'Accuracy on validation set: {evaluation_result[1]}')

13/13 [==============================] - 12s 912ms/step - loss: 0.4655 - accuracy: 0.9075
Loss on validation set: 0.4655390679836273
Accuracy on validation set: 0.9075000286102295


In [10]:
# Função para criar o segundo modelo CNN
def create_second_cnn_model():
    model = Sequential()
    model.add(Conv2D(64, (3, 3), input_shape=input_shape, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [11]:
# Treina o segundo modelo CNN
model2 = create_second_cnn_model()
history2 = model2.fit(load_images_with_error_handling(train_generator),
                      validation_data=load_images_with_error_handling(validation_generator),
                      epochs=10)

Epoch 1/10
     17/Unknown - 129s 7s/step - loss: 6.7987 - accuracy: 0.3290Error loading batch. Skipping...
     93/Unknown - 694s 7s/step - loss: 1.8969 - accuracy: 0.6247Error loading batch. Skipping...
    141/Unknown - 1052s 7s/step - loss: 1.4231 - accuracy: 0.6968Error loading batch. Skipping...
    172/Unknown - 1283s 7s/step - loss: 1.2752 - accuracy: 0.7148Error loading batch. Skipping...
    239/Unknown - 1779s 7s/step - loss: 1.0145 - accuracy: 0.7649Error loading batch. Skipping...
    289/Unknown - 2152s 7s/step - loss: 0.8858 - accuracy: 0.7913Error loading batch. Skipping...
    335/Unknown - 2495s 7s/step - loss: 0.7981 - accuracy: 0.8079Error loading batch. Skipping...
    393/Unknown - 2925s 7s/step - loss: 0.7074 - accuracy: 0.8283Error loading batch. Skipping...
    457/Unknown - 3395s 7s/step - loss: 0.6305 - accuracy: 0.8446Error loading batch. Skipping...
    504/Unknown - 3741s 7s/step - loss: 0.5843 - accuracy: 0.8547Error loading batch. Skipping...
    547/Unk

KeyboardInterrupt: ignored

In [12]:
# Avaliação do segundo modelo no conjunto de validação
evaluation_result2 = model2.evaluate(validation_generator)
print(f'Loss on validation set (Model 2): {evaluation_result2[0]}')
print(f'Accuracy on validation set (Model 2): {evaluation_result2[1]}')

13/13 [==============================] - 29s 2s/step - loss: 0.4184 - accuracy: 0.8950
Loss on validation set (Model 2): 0.41843730211257935
Accuracy on validation set (Model 2): 0.8949999809265137


In [13]:
# Carrega a base do modelo VGG16 sem as camadas densas (fully connected)
base_model_vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

# Congela as camadas do modelo base
for layer in base_model_vgg16.layers:
    layer.trainable = False

# Adiciona camadas densas personalizadas para a classificação
model_vgg16 = Sequential([
    base_model_vgg16,
    Flatten(),
    Dense(256, activation='relu'),
    Dense(classes, activation='softmax')
])

# Compila o modelo
model_vgg16.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Treina o modelo com Transfer Learning
history_vgg16 = model_vgg16.fit(load_images_with_error_handling(train_generator),
                                validation_data=load_images_with_error_handling(validation_generator),
                                epochs=10)

58889256/58889256 [==============================] - 0s 0us/step
Epoch 1/10
     48/Unknown - 844s 18s/step - loss: 1.3754 - accuracy: 0.7409Error loading batch. Skipping...
     84/Unknown - 1477s 18s/step - loss: 0.8638 - accuracy: 0.8207

KeyboardInterrupt: ignored

In [14]:
# Carrega a base do modelo ResNet50 sem as camadas densas (fully connected)
base_model_resnet50 = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

# Congela as camadas do modelo base
for layer in base_model_resnet50.layers:
    layer.trainable = False

# Adiciona camadas densas personalizadas para a classificação
model_resnet50 = Sequential([
    base_model_resnet50,
    Flatten(),
    Dense(256, activation='relu'),
    Dense(classes, activation='softmax')
])

# Compila o modelo
model_resnet50.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Treina o modelo com Transfer Learning
history_resnet50 = model_resnet50.fit(load_images_with_error_handling(train_generator),
                                      validation_data=load_images_with_error_handling(validation_generator),
                                      epochs=10)

94765736/94765736 [==============================] - 0s 0us/step
Epoch 1/10
      5/Unknown - 38s 6s/step - loss: 22.9348 - accuracy: 0.2500Error loading batch. Skipping...
     57/Unknown - 368s 6s/step - loss: 7.2078 - accuracy: 0.3514Error loading batch. Skipping...
     88/Unknown - 568s 6s/step - loss: 5.1556 - accuracy: 0.4126Error loading batch. Skipping...
    130/Unknown - 835s 6s/step - loss: 3.9322 - accuracy: 0.4464Error loading batch. Skipping...
    189/Unknown - 1210s 6s/step - loss: 3.0360 - accuracy: 0.4947Error loading batch. Skipping...
    220/Unknown - 1409s 6s/step - loss: 2.7746 - accuracy: 0.5071Error loading batch. Skipping...
    266/Unknown - 1703s 6s/step - loss: 2.4697 - accuracy: 0.5282Error loading batch. Skipping...
    307/Unknown - 1967s 6s/step - loss: 2.2684 - accuracy: 0.5428Error loading batch. Skipping...
    380/Unknown - 2432s 6s/step - loss: 1.9932 - accuracy: 0.5695Error loading batch. Skipping...
    404/Unknown - 2587s 6s/step - loss: 1.9266

KeyboardInterrupt: ignored

In [15]:
# Avaliação dos modelos no conjunto de validação
evaluation_result2 = model2.evaluate(validation_generator)
evaluation_result_vgg16 = model_vgg16.evaluate(validation_generator)
evaluation_result_resnet50 = model_resnet50.evaluate(validation_generator)

print(f'\nEvaluation Results for Model 2:')
print(f'Loss: {evaluation_result2[0]}, Accuracy: {evaluation_result2[1]}')

print(f'\nEvaluation Results for VGG16 Transfer Learning Model:')
print(f'Loss: {evaluation_result_vgg16[0]}, Accuracy: {evaluation_result_vgg16[1]}')

print(f'\nEvaluation Results for ResNet50 Transfer Learning Model:')
print(f'Loss: {evaluation_result_resnet50[0]}, Accuracy: {evaluation_result_resnet50[1]}')

13/13 [==============================] - 79s 6s/step - loss: 0.8724 - accuracy: 0.6600

Evaluation Results for Model 2:
Loss: 0.42671453952789307, Accuracy: 0.887499988079071

Evaluation Results for VGG16 Transfer Learning Model:
Loss: 0.24068640172481537, Accuracy: 0.9075000286102295

Evaluation Results for ResNet50 Transfer Learning Model:
Loss: 0.8724290728569031, Accuracy: 0.6600000262260437


In [19]:
# Função para plotar resultados de treinamento
def plot_training_results(history, title):
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'{title} - Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'{title} - Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.show()